In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

1. Load in the data

In [ ]:
aec = pd.read_csv("../data/aec-sirindhorn.csv")
sarea = pd.read_csv("../data/area-sirindhorn.csv", parse_dates=['system:time_start']).drop(['system:index', '.geo'], axis=1).rename({'system:time_start': 'date'}, axis=1)

# cleaning has to be done
sarea.loc[sarea['water_area'] == -9999, 'water_area'] = np.nan
sarea.loc[sarea['corrected_area'] == -9999, 'corrected_area'] = np.nan
sarea.loc[:, 'corrected_area'] = sarea.loc[:, 'corrected_area'].fillna(method='ffill')

# CODE FOR PLOTTING BELOW (ignore)
f = make_subplots(
    rows=1, cols=2,
    subplot_titles=("AEC", "Surface Area"),
    horizontal_spacing=0.05,
    shared_yaxes=True)

f.add_trace(go.Scatter(
    x = aec['Elevation'],
    y = aec['Area'],
    mode = 'markers+lines',
    name = 'AEC - Sirindhorn'
), row=1, col=1)

f.add_trace(go.Scatter(
    x = sarea['date'],
    y = sarea['corrected_area'],
    mode = 'markers+lines',
    name = 'Surface area (TMS-OS)'
), row=1, col=2)

f.update_layout(
    title_text = 'Sirindhorn - AEC and Surface Areas', title_x = 0.5,
    legend = dict(
        yanchor = 'top',
        y = 0.99,
        xanchor = 'left',
        x = 0.01
    ),
    margin = dict(l=50, r=50, t=100, b=50)
)

f.update_xaxes(title_text = "Water Level (m)", row=1, col=1)
f.update_yaxes(title_text = "Surface Area (km^2)", row=1, col=1)
f.update_xaxes(title_text = "Date", row=1, col=2)
f.update_yaxes(title_text = "Surface Area (km^2)", row=1, col=2)

2. Convert from _Areas_ to _Water Level_, and perform calculation using the trapezoidal method - 
$$\Delta S = \frac{1}{2} \times (A_t + A_{t-1}) \times (h_t - h_{t-1})$$

In [ ]:
df = sarea.copy()

# Define a function using the AEC for converting Area -> Elevation 
get_elev = lambda area: np.interp(area, aec['Area'], aec['Elevation'])

# Convert Area -> Elevation
df['wl (m)'] = df['corrected_area'].apply(get_elev)

# Convert area from km^2 to m^2
df['area (m2)'] = df['corrected_area'] * 1e6

# Perform the calculation using Trapezoidal method
A0 = df['area (m2)'].iloc[:-1]
A1 = df['area (m2)'].iloc[1:]

h0 = df['wl (m)'].iloc[:-1]
h1 = df['wl (m)'].iloc[1:]

S = (h1.values - h0.values)*(A1.values + A0.values)/2
S = np.insert(S, 0, np.nan)                     # Insert a Not-A-Number value at the beginning 

df['dS'] = S

Let's plot the ∆S now

In [ ]:
# CODE FOR PLOTTING BELOW (ignore)
f = make_subplots(
    rows=2, cols=1,
    subplot_titles=("∆S", "Surface Areaa"),
    vertical_spacing=0.1,
    shared_xaxes=True)

f.add_trace(go.Scatter(
    x = df['date'],
    y = df['dS']*1e-6,  # convert to Million m^3
    mode = 'markers+lines',
    name = 'Estimated ∆S - Sirindhorn'
), row=1, col=1)

f.add_trace(go.Scatter(
    x = df['date'],
    y = df['corrected_area'],
    mode = 'markers+lines',
    name = 'Surface area (TMS-OS)'
), row=2, col=1)

f.update_layout(
    title_text = 'Sirindhorn - ∆S and corresponding Surface Areas', title_x = 0.5,
    legend = dict(
        yanchor = 'top',
        y = 0.99,
        xanchor = 'left',
        x = 0.01
    ),
    margin = dict(l=50, r=50, t=100, b=50)
)

f.update_yaxes(title_text = "∆S (Mil. m^3)", row=1, col=1)
f.update_xaxes(title_text = "Date", row=2, col=1)
f.update_yaxes(title_text = "Surface Area (km^2)", row=2, col=1)

In [ ]:
df.to_csv("../data/dels-sirindhorn.csv", index=False)